# ***Main Function: Lab incubation Simulation - MCMC paras***

# ***Sites selection***

In [2]:
#import the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import scipy
import time
import math
import sklearn
import cv2 as cv
import matplotlib.pyplot as plt

from scipy import stats, integrate
from scipy.signal import find_peaks
from collections import Counter

#%matplotlib inline

In [3]:
###Peak Sites###
Lignin_LowPeak  = ['GRSM3','GRSM4','LENO1','OSBS1','OSBS2','SJER3','WREF2','WREF4']
Lignin_HighPeak = ['HARV1','HARV4','LENO2','LENO3','LENO4','SCBI1','UNDE3']
Litter_peak     = ['PUUM2','PUUM3','SCBI2','SCBI3','SCBI4'] 

# LITTER #
# PUUM 1,2,3,4    # Litter peak
# SCBI 2,3,4      # litter peaks
# LIGNIN #
#'LENO2'          # lignin peak, but can capture
# OSBS3,4         # no lignin peak
# SJER2           # no-lignin peak

# get all site name #
sitename = ['BONA','CPER','DSNY','GRSM','HARV','KONZ','LENO','NIWO','ONAQ','OSBS','PUUM','SCBI','SJER','SRER','TALL','TOOL','UNDE','WOOD','WREF','YELL']
sitenum  = 4
sample_site_index = []
for i in range(len(sitename)):
    for j in range(sitenum):
        sample_site_index.append("{Site}{Num}".format(Site=sitename[i],Num=j+1))
        
### Valid Sites: Exclude peak sites ###
validSite = Counter(sample_site_index)-Counter(Lignin_LowPeak)-Counter(Lignin_HighPeak)-Counter(Litter_peak)
validSite = list(validSite.elements())
print((validSite),len(validSite))

### Peak Sites: ###
peakSite = Lignin_LowPeak + Lignin_HighPeak
print(peakSite,len(peakSite))

peakSite_order  = ['GRSM3', 'GRSM4', 'HARV1', 'HARV4', 'LENO1','LENO2', 'LENO3', 'LENO4',  'OSBS1', 'OSBS2', 'SCBI1', 'SJER3','UNDE3','WREF2', 'WREF4' ]
nopeakSite_order = ['BONA1','BONA2','BONA3','BONA4','CPER1','CPER2','CPER3','CPER4','DSNY1','DSNY2','DSNY3','DSNY4','GRSM1','GRSM2','HARV2','HARV3','KONZ1','KONZ2','KONZ3','KONZ4','NIWO1','NIWO2',
                    'NIWO3','NIWO4','ONAQ1','ONAQ2','ONAQ3','ONAQ4','OSBS3','OSBS4','PUUM1','PUUM2','PUUM3','PUUM4','SCBI2','SCBI3','SCBI4','SJER1','SJER2','SJER4','SRER1','SRER2','SRER3','SRER4',
                    'TALL1','TALL2','TALL3','TALL4','TOOL1','TOOL2','TOOL3','TOOL4','UNDE1','UNDE2','UNDE4','WOOD1','WOOD2','WOOD3','WOOD4','WREF1','WREF3','YELL1','YELL2','YELL3','YELL4']

### Random choose sites ###
#sample_sitetest_index = random.sample(sample_site_index,int(len(sample_site_index)*0.7))


### M2 layer ###
peakSite_M2_check = ['UNDE4','TALL4']
peakSite_order_M2   = ['GRSM4','HARV1','HARV4','LENO1','LENO2','LENO3','LENO4','OSBS2','OSBS3','OSBS4','SCBI1','SCBI3','SJER2','TALL1','TALL2','TALL3','TOOL1','TOOL2','TOOL3','TOOL4','UNDE1','UNDE3','WREF1','WREF2','WREF4']
print(len(peakSite_order_M2))
nopeakSite_order_M2 = list((Counter(sample_site_index) - Counter(peakSite_order_M2)-Counter(['KONZ1','KONZ2','KONZ3','KONZ4'])).elements())
print(len(nopeakSite_order_M2))

['BONA1', 'BONA2', 'BONA3', 'BONA4', 'CPER1', 'CPER2', 'CPER3', 'CPER4', 'DSNY1', 'DSNY2', 'DSNY3', 'DSNY4', 'GRSM1', 'GRSM2', 'HARV2', 'HARV3', 'KONZ1', 'KONZ2', 'KONZ3', 'KONZ4', 'NIWO1', 'NIWO2', 'NIWO3', 'NIWO4', 'ONAQ1', 'ONAQ2', 'ONAQ3', 'ONAQ4', 'OSBS3', 'OSBS4', 'PUUM1', 'PUUM4', 'SJER1', 'SJER2', 'SJER4', 'SRER1', 'SRER2', 'SRER3', 'SRER4', 'TALL1', 'TALL2', 'TALL3', 'TALL4', 'TOOL1', 'TOOL2', 'TOOL3', 'TOOL4', 'UNDE1', 'UNDE2', 'UNDE4', 'WOOD1', 'WOOD2', 'WOOD3', 'WOOD4', 'WREF1', 'WREF3', 'YELL1', 'YELL2', 'YELL3', 'YELL4'] 60
['GRSM3', 'GRSM4', 'LENO1', 'OSBS1', 'OSBS2', 'SJER3', 'WREF2', 'WREF4', 'HARV1', 'HARV4', 'LENO2', 'LENO3', 'LENO4', 'SCBI1', 'UNDE3'] 15
25
51


# ***MCMC Posterior***

In [4]:
para_name = ['CN_AOM1', 'CN_AOM2', 'CN_AOM3', 'CN_SMB1', 'CN_SMB2', 'CN_SMR', 'CN_NOM', 'CN_MOM',   
            'k1max_AOM1','k2max_AOM1', 'k1max_AOM2', 'k2max_AOM2', 'k1max_AOM3', 'k2max_AOM3', 'k1max_NOM', 'k2max_NOM', 'kmax_NOM', 'kmax_SMR', 'kmax_MOM',   
            'CUE1_AOM1', 'CUE1_AOM2', 'CUE1_AOM3', 'CUE1_NOM', 'CUE2_AOM1', 'CUE2_AOM2', 'CUE2_AOM3', 'CUE2_NOM',   
            'KM1_AOM1', 'KM1_AOM2', 'KM1_AOM3', 'KM1_NOM', 'KM2_AOM1', 'KM2_AOM2', 'KM2_AOM3', 'KM2_NOM', 'death1_rate', 'death2_rate', 'death2_rate', 'lig_p1', 'lig_p2']

In [8]:
import os
import fnmatch 
import scipy.io
import pandas as pd
for site in peakSite:
    in_path = r'C:\Users\boyi\Desktop\8.MCMC_Uniform_ForLabIcubation_LoopAllSite_MM\20200210_dream_zs_restart_with_SOM\MAMC_Result_MM_KM_Calibration\20220712_POMMAOM'
    mat  = scipy.io.loadmat(in_path+'\{site_name}.mat'.format(site_name=site))
    mat.keys()
    post = pd.DataFrame(mat['Z0'])
    out_path = r'C:\Users\boyi\Desktop\8.MCMC_Uniform_ForLabIcubation_LoopAllSite_MM\20200210_dream_zs_restart_with_SOM\MAMC_Result_MM_KM_Calibration\20220712_POMMAOM'
    post.to_csv(out_path + '\{site_name}_lss_para_posterior.csv'.format(site_name=site),header=para_name,index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\boyi\\Desktop\\8.MCMC_Uniform_ForLabIcubation_LoopAllSite_MM\\20200210_dream_zs_restart_with_SOM\\MAMC_Result_MM_KM_Calibration\\20220712_POMMAOM\\NIWO1.mat'

# ***Main Function: (11/10/2021)***

In [12]:
##Sample site & Lab incubation days## 

day  = 571+10
para_sample = 200

#Package import#
import numpy as np
import pandas as pd
import time
from Decomposition_MM import CWD_pool
from Decomposition_MM import SOM_pool
from Decomposition_MM import av_Nitro
from Decomposition_MM import CWDFrag_OrgInAllocLab		       ###QQQ:20210602
from Decomposition_MM import SoilDecom_pot
from Decomposition_MM import SoilDecom_real
from Decomposition_MM import initial_content
from Decomposition_MM import lls_CO2


#Looping for Each Site#

for sample_site in peakSite:
    print(sample_site)
    Rh_total  = np.zeros((para_sample,day-1))
    Rh_litter = np.zeros((para_sample,day-1))
    Rh_lignin = np.zeros((para_sample,day-1))
    Rh_soil   = np.zeros((para_sample,day-1))
    AOM1_C    = np.zeros((para_sample,day-1))
    AOM2_C    = np.zeros((para_sample,day-1))
    AOM3_C    = np.zeros((para_sample,day-1))
    AOM1_N    = np.zeros((para_sample,day-1))
    AOM2_N    = np.zeros((para_sample,day-1))
    AOM3_N    = np.zeros((para_sample,day-1))
    Soil_N    = np.zeros((para_sample,day-1))
    for i in range(0,para_sample):
        #Input data#
        obser_data = pd.read_csv("./OBSER_data_POMMAOM.csv",index_col=0)
        #Fixed Paras#
        # decom_para = pd.read_csv("decom_para.csv",index_col=0)
        #Optimized para from MCMC#
        decom_para_posterior = pd.read_csv(r"C:\Users\boyi\Desktop\8.MCMC_Uniform_ForLabIcubation_LoopAllSite_MM\20200210_dream_zs_restart_with_SOM\MAMC_Result_MM_KM_Calibration\20220712_POMMAOM\{Site_Num}_lss_para_posterior.csv".format(Site_Num=sample_site),index_col=False)
        decom_para_post      = decom_para_posterior.iloc[i,:]
        #Optimized parameters#
        CN_AOM1	   = decom_para_post[0]
        CN_AOM2	   = decom_para_post[1]
        CN_AOM3	   = decom_para_post[2]
        CN_SMB1	   = decom_para_post[3]
        CN_SMB2	   = decom_para_post[4]
        CN_SMR	   = decom_para_post[5]
        CN_NOM	   = decom_para_post[6]
        CN_MOM	   = decom_para_post[7]
        k1max_AOM1 = decom_para_post[8]
        k2max_AOM1 = decom_para_post[9]
        k1max_AOM2 = decom_para_post[10]
        k2max_AOM2 = decom_para_post[11]
        k1max_AOM3 = decom_para_post[12]
        k2max_AOM3 = decom_para_post[13]
        k1max_NOM  = decom_para_post[14]
        k2max_NOM  = decom_para_post[15]
        kmax_NOM   = decom_para_post[16]
        kmax_SMR   = decom_para_post[17]
        kmax_MOM   = decom_para_post[18]
        CUE1_AOM1  = decom_para_post[19]
        CUE1_AOM2  = decom_para_post[20]
        CUE1_AOM3  = decom_para_post[21]
        CUE1_NOM   = decom_para_post[22]
        CUE2_AOM1  = decom_para_post[23]
        CUE2_AOM2  = decom_para_post[24]
        CUE2_AOM3  = decom_para_post[25]
        CUE2_NOM   = decom_para_post[26]
        KM1_AOM1   = decom_para_post[27]
        KM1_AOM2   = decom_para_post[28]
        KM1_AOM3   = decom_para_post[29]
        KM1_NOM    = decom_para_post[30]
        KM2_AOM1   = decom_para_post[31]
        KM2_AOM2   = decom_para_post[32]
        KM2_AOM3   = decom_para_post[33]
        KM2_NOM    = decom_para_post[34]
        death1_rate1 = decom_para_post[35]
        death2_rate1 = decom_para_post[36]
        death2_rate2 = decom_para_post[37]
        lig_p1	   = decom_para_post[38]
        lig_p2	   = decom_para_post[39]
        #Class initialization#
        litter = CWD_pool(obser_data,sample_site,"litter")
        AOM1   = SOM_pool(obser_data,sample_site,'AOM1')
        AOM2   = SOM_pool(obser_data,sample_site,'AOM2')
        AOM3   = SOM_pool(obser_data,sample_site,'AOM3')
        SMB1   = SOM_pool(obser_data,sample_site,'SMB1')
        SMB2   = SOM_pool(obser_data,sample_site,'SMB2')
        SMR    = SOM_pool(obser_data,sample_site,'SMR')
        NOM    = SOM_pool(obser_data,sample_site,'NOM')
        MOM    = SOM_pool(obser_data,sample_site,'MOM')
        avN    = av_Nitro(obser_data,sample_site)
        initCont   = initial_content()
        initCont.MicroSMB1Zero(SMB1.C)
        initCont.MicroSMB2Zero(SMB2.C)
        #initCont.LitterSub(litter.C*kbase_litter)
        #initCont.SoilSub(NOM.C*kmax_NOM)
        #Data Collection#
        Rh=[]
        flag_SMB1 = 0
        flag_SMB2 = 0
        
        CWDFrag_OrgInAllocLab(sample_site,obser_data,litter,AOM1,AOM2,AOM3,CN_AOM1,CN_AOM2,CN_AOM3)

        #Looping for Each Inbubation Day#
        for t in range(1,day):
            potimm = 0
            Nmin = 0
            litligsoil_CO2 = lls_CO2()

            bimm_AOM1,bimm_AOM2,bimm_AOM3,bimm_SMR,bimm_NOM,bimm_MOM,potimm = SoilDecom_pot(sample_site,t,obser_data,potimm,flag_SMB1,flag_SMB2,lig_p1,lig_p2,             #Scalar(6)#   ###QQQ: 20210908: lignin decay log para #
                  																		 AOM1,AOM2,AOM3,SMB1,SMB2,SMR,NOM,MOM,initCont,                   #Class(9)#
                  																		 CN_SMB1,CN_SMB2,CN_SMR,CN_NOM,CN_MOM,                            #CN Ratio(5)#
                  																		 k1max_AOM1,k1max_AOM2,k1max_AOM3,k1max_NOM,k2max_AOM1,k2max_AOM2,k2max_AOM3,k2max_NOM,kmax_SMR,kmax_NOM,kmax_MOM,        #Decay Rate(6)#
                  																		 CUE1_AOM1,CUE1_AOM2,CUE1_AOM3,CUE1_NOM,CUE2_AOM1,CUE2_AOM2,CUE2_AOM3,CUE2_NOM,    #Fraction(6)#
                  																		 KM1_AOM1,KM1_AOM2,KM1_AOM3,KM1_NOM,KM2_AOM1,KM2_AOM2,KM2_AOM3,KM2_NOM,death1_rate1,death2_rate1,death2_rate2)

            Rh,flag_SMB1,flag_SMB2 = SoilDecom_real (sample_site,t,obser_data,Nmin,potimm,flag_SMB1,flag_SMB2,Rh,lig_p1,lig_p2,     #Scalar(7)#  ###QQQ: 20210908: add 2 logistic lignin para to control lignin decomposition ###
                                                    AOM1,AOM2,AOM3,SMB1,SMB2,SMR,NOM,MOM,avN,initCont,litligsoil_CO2,              #Class(10)#
                                                    bimm_AOM1,bimm_AOM2,bimm_AOM3,bimm_SMR,bimm_NOM,bimm_MOM,                      #N scalar(6)#
                                                    CN_SMB1,CN_SMB2,CN_SMR,CN_NOM,CN_MOM,                                          #CN Ratio#
                                                    k1max_AOM1,k1max_AOM2,k1max_AOM3,kmax_SMR,k1max_NOM,kmax_NOM,kmax_MOM,k2max_AOM1,k2max_AOM2,k2max_AOM3,k2max_NOM,        #Decay Rate(6)#
                                                    CUE1_AOM1,CUE1_AOM2,CUE1_AOM3,CUE1_NOM,CUE2_AOM1,CUE2_AOM2,CUE2_AOM3,CUE2_NOM,
                                                    KM1_AOM1,KM1_AOM2,KM1_AOM3,KM1_NOM,KM2_AOM1,KM2_AOM2,KM2_AOM3,KM2_NOM,death1_rate1,death2_rate1,death2_rate2)



        ##Data Output##
        Rh_array = np.array(Rh)
        Rh_day = Rh_array.reshape(-1,12)
        Rh_total[i,:]  = Rh_day[:,-4]
        Rh_litter[i,:] = Rh_day[:,-3]
        Rh_lignin[i,:] = Rh_day[:,-2]
        Rh_soil[i,:]   = Rh_day[:,-1]
              
#         Cpool_array = np.array(Cpool)
#         Cpool_day   = Cpool_array.reshape(-1,17)
#         AOM1_C[i,:] = Cpool_day[:,0]
#         AOM2_C[i,:] = Cpool_day[:,1]
#         AOM3_C[i,:] = Cpool_day[:,2]
#         AOM1_N[i,:] = Cpool_day[:,8]
#         AOM2_N[i,:] = Cpool_day[:,9]
#         AOM3_N[i,:] = Cpool_day[:,10]
#         Soil_N[i,:] = Cpool_day[:,16]
    
    out_path = r"C:\Users\boyi\Desktop\8.MCMC_Uniform_ForLabIcubation_LoopAllSite_MM\20200210_dream_zs_restart_with_SOM\MAMC_Result_MM_KM_Calibration\20220712_POMMAOM"
    np.savetxt(out_path + '/{Site_Num}_lss_total.csv'.format(Site_Num=sample_site),Rh_total,delimiter=",")
    np.savetxt(out_path + '/{Site_Num}_lss_litter.csv'.format(Site_Num=sample_site),Rh_litter,delimiter=",")
    np.savetxt(out_path + '/{Site_Num}_lss_lignin.csv'.format(Site_Num=sample_site),Rh_lignin,delimiter=",")
    np.savetxt(out_path + '/{Site_Num}_lss_soil.csv'.format(Site_Num=sample_site),Rh_soil,delimiter=",")
    
    
    
#     np.savetxt('./modelResult_lss/modelResult_20220330/{Site_Num}_AOM1_pool.csv'.format(Site_Num=sample_site),AOM1_C,delimiter=",")
#     np.savetxt('./modelResult_lss/modelResult_20220330/{Site_Num}_AOM2_pool.csv'.format(Site_Num=sample_site),AOM2_C,delimiter=",")
#     np.savetxt('./modelResult_lss/modelResult_20220330/{Site_Num}_AOM3_pool.csv'.format(Site_Num=sample_site),AOM3_C,delimiter=",")
#     np.savetxt('./modelResult_lss/modelResult_20220330/{Site_Num}_AOM1_Npool.csv'.format(Site_Num=sample_site),AOM1_N,delimiter=",")
#     np.savetxt('./modelResult_lss/modelResult_20220330/{Site_Num}_AOM2_Npool.csv'.format(Site_Num=sample_site),AOM2_N,delimiter=",")
#     np.savetxt('./modelResult_lss/modelResult_20220330/{Site_Num}_AOM3_Npool.csv'.format(Site_Num=sample_site),AOM3_N,delimiter=",")
#     np.savetxt('./modelResult_lss/modelResult_20220330/{Site_Num}_Soil_Npool.csv'.format(Site_Num=sample_site),Soil_N,delimiter=",")

GRSM3
0.0048590000000000005 -6.476308490945983 35.8221384
15.4848347069188 1 2.4780568737953645
0.428398801455777 1.2634091180023863 3.2946219262332246
0.7124749061421419 1.2039692871471666 3.5566508835993207
0.9456635719904691 1.1360545414012724 3.8394763962895566
1.2181142412986194 1.0657756948548844 4.144794353381294
1.5216148283734914 1 4.474393221005946
1.8471323484446305 1 4.830204317186055
2.315071640549915 1 5.214339817446913
2.8283142907812726 1 5.629035188301603
3.326411409069393 1 6.076719957357778
0.034486329211222674 1.4922356890731103 6.559900315479199
0.05166651362367334 1.4892159746061082 3.5413517539648307
0.051585925402024085 1.4800737070006398 3.682144933683878
0.05129848868449101 1.4809410604309277 3.828531775460763
0.051363606130820694 1.4816452752682674 3.980744504190225
0.05142881985536957 1.482323535779266 4.139008549320117
0.051498912378927716 1.4829751861622291 4.303564571741373
0.05157385410009458 1.483601338236161 4.474662716078462
0.051653647131645455 1.484

In [ ]:
1

1